![xwidgets](../src/xwidgets.svg)

<center> <h1>Widgets in C++ with xeus and cling</h1> </center>

## Button widget

In [ ]:
#include <iostream>
#include "xwidgets/xbutton.hpp"

In [ ]:
xw::button button;

In [ ]:
void foo()
{
    std::cout << "Clicked!" << std::endl;
}

In [ ]:
button.on_click(foo);
    
button.display()

In [ ]:
button.description = "button";

## Numerical input

In [ ]:
#include "xwidgets/xnumeral.hpp"

In [ ]:
xw::numeral<double> numeral;
numeral.display()

In [ ]:
numeral.value = 4;

In [ ]:
numeral.value()

## Image

In [ ]:
#include "xwidgets/ximage.hpp"
#include "xwidgets/xprogress.hpp"

In [ ]:
auto input = xw::image_from_file("my_awesome_cat.jpg").finalize();
input

![xtensor-fftw](../src/xtensor-fftw.svg)

In [ ]:
#pragma cling(optimize, 3)

#include <string>
#include <fstream>
#include <complex>

#include "xtl/xbase64.hpp"

#include "xtensor/xarray.hpp"
#include "xtensor/xview.hpp"
#include "xtensor/xbuilder.hpp"

#include "xtensor-io/ximage.hpp"

#include "xtensor-fftw/basic.hpp"
#include "xtensor-fftw/helper.hpp"

In [ ]:
std::vector<char> read_file(const char* filename)
{
    std::basic_ifstream<char> file(filename, std::ios::binary);
    return std::vector<char>((std::istreambuf_iterator<char>(file)), std::istreambuf_iterator<char>());
}

In [ ]:
template <class E>
std::vector<char> to_png_buffer(const xt::xexpression<E>& e)
{
    const char* temp_filename = "/tmp/xio_image.png";
    xt::dump_image(temp_filename, e);
    return read_file(temp_filename);
}

In [ ]:
xw::progress<double> progress;
progress.description = "Progress:";
progress

In [ ]:
auto output = xw::image();

xt::xarray<double> image_R;
xt::xarray< std::complex<double> > d_image_dx_fs_bw;
xt::xarray< std::complex<double> > d_image_dy_fs_bw;

image_R = xt::view(xt::load_image("my_awesome_cat.jpg"), xt::all(), xt::all(), 0);
progress.value = 10;

auto image_fs_bw = xt::fftw::rfft2(image_R);
progress.value = 20;

std::complex<double> i {0, 1};
d_image_dx_fs_bw = i * xt::view(xt::fftw::fftscale<double>(image_R.shape()[0]), xt::all(), xt::newaxis()) * image_fs_bw;
progress.value = 30;
d_image_dy_fs_bw = i * xt::fftw::rfftscale<double>(image_R.shape()[1]) * image_fs_bw;
progress.value = 40;

auto d_image_dx_bw = xt::fftw::irfft2(d_image_dx_fs_bw);
progress.value = 50;
auto d_image_dy_bw = xt::fftw::irfft2(d_image_dy_fs_bw);
progress.value = 60;

auto d_image_grad_bw = xt::sqrt(d_image_dx_bw * d_image_dx_bw + d_image_dy_bw * d_image_dy_bw);
progress.value = 70;

double amax_d_image_grad_bw3 = xt::amax(d_image_grad_bw)[0];
progress.value = 80;

auto res = xt::cast<unsigned char>(255 - (d_image_grad_bw / amax_d_image_grad_bw3 * 255));
progress.value = 90;
output.value = to_png_buffer(res);
progress.value = 100;

output